In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
from datetime import datetime
from tqdm import tqdm

# ffmpeg installation location, for creating videos
plt.rcParams['animation.ffmpeg_path'] = str('/usr/bin/ffmpeg')
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
from ipywidgets import Button

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

%load_ext autoreload
%autoreload 2

# StyleGAN2 Repo
sys.path.append('/tf/notebooks/stylegan2')

# StyleGAN Utils
from stylegan_utils import load_network, gen_image_fun, synth_image_fun, create_video
# v1 override
from stylegan_utils import load_network_v1 as load_network
from stylegan_utils import gen_image_fun_v1 as gen_image_fun
from stylegan_utils import synth_image_fun_v1 as synth_image_fun

import run_projector
import projector
import training.dataset
import training.misc

# Data Science Utils
sys.path.append(os.path.join(*[os.pardir]*3, 'data-science-learning'))

from ds_utils import generative_utils

In [ ]:
res_dir = Path('/tf/media/datasets/stylegan/result_dir')

# Load Network

In [ ]:
MODELS_DIR = res_dir
MODEL_NAME = '00008-sgan-dress512-2gpu'
SNAPSHOT_NAME = 'network-snapshot-010742'

Gs, Gs_kwargs, _ = load_network(str(MODELS_DIR / MODEL_NAME / SNAPSHOT_NAME) + '.pkl')

Z_SIZE = Gs.input_shape[1]
IMG_SIZE = Gs.output_shape[2:]
IMG_SIZE

In [ ]:
img = gen_image_fun(Gs, np.random.randn(1, Z_SIZE), Gs_kwargs, False, truncation_psi=1)
plt.imshow(img)

# Explore Directions

In [ ]:
def plot_direction_grid(dlatent, direction, coeffs):
    fig, ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=100)
    
    for i, coeff in enumerate(coeffs):
        new_latent = (dlatent.copy() + coeff*direction)
        ax[i].imshow(synth_image_fun(Gs, new_latent, Gs_kwargs, randomize_noise=False))
        ax[i].set_title(f'Coeff: {coeff:0.1f}')
        ax[i].axis('off')
    plt.show()

In [ ]:
# load learned direction
direction = np.load('/tf/media/datasets/stylegan/learned_directions.npy')

In [ ]:
nb_latents = 5

# generate dlatents from mapping network
dlatents = Gs.components.mapping.run(np.random.rand(nb_latents, Z_SIZE), None, truncation_psi=1.)

for i in range(nb_latents):
    plot_direction_grid(dlatents[i:i+1], direction, np.linspace(-2, 2, 5))

## Interactive

In [ ]:
# Setup plot image
dpi = 100
fig, ax = plt.subplots(dpi=dpi, figsize=(IMG_SIZE[0]/dpi, IMG_SIZE[1]/dpi))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0, wspace=0)
plt.axis('off')
im = ax.imshow(gen_image_fun(Gs, np.random.randn(1, Z_SIZE), Gs_kwargs, False, truncation_psi=1))

#prevent any output for this cell
plt.close()

In [ ]:
%matplotlib inline

dlatents = Gs.components.mapping.run(np.random.rand(nb_latents, Z_SIZE), None, truncation_psi=0.5)
latent_vector = dlatents[0:1]

@interact
def i_direction(coeff = (-2., 2.)):
    new_latent_vector = latent_vector.copy()
    new_latent_vector = (latent_vector + coeff*direction)
    im.set_data(synth_image_fun(Gs, new_latent_vector, Gs_kwargs, False))
    ax.set_title('Coeff: %0.1f' % coeff)
    display(fig)